# Demonstration of the usage of vita-python utilities
This notebook demonstrate three major usages:
1. Define area using numpy 3d array and save it as `.vtk` file for VItA vessel synthesis
2. Synthesize vessels based on docker containerized VItA
3. Voxelize the generated `vessel.vtp` file to produce the 3d numpy.ndarray

For more information about the `VItA` and `vita docker container`,\
Please check the `README.md` of this repo.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Define vessel synthesis area

In [2]:
from vita_python_utils import random_index_of_one, save_np_vtk

In [3]:
## Create the volume with a sphere
x, y, z = np.ogrid[:50, :50, :50]
dx = 1
dist_sq = (x - 25)**2 + (y - 25)**2 + (z - 25)**2
sphere_mask = dist_sq <= 20**2

In [4]:
## Save the volume as vtk file
save_path = 'data/sphere'
final_vtk_path = save_np_vtk(sphere_mask, save_path, spacing=dx, smooth=True, filetype='vtk')

Running marching cubes to generate mesh...
The mesh is watertight (closed).
Original mesh: 7542 vertices.
Smoothing the generated mesh using `trimesh`...
Smoothed mesh: 7542 vertices.
Workflow complete. The file is ready. Saved as 'data/sphere.vtk'


In [6]:
## Define the vessel origin
ves_pos = random_index_of_one(sphere_mask)*dx
print(ves_pos)

(15, 23, 40)


## Using VItA to generate vessel

In [ ]:
# prepare docker commands
num_ves = 10
prefix = 'sphere'
docker_com = gen_vita_docker_com(ves_pos, num_ves,
                                 gen_prefix=prefix,
                                 vtk_name=prefix+'.vtk',
                                 vita_path=vita_folder+"vita_headphantom")